In [99]:
import datetime
import time
import calcTimeStamp
from datetime import timezone
from datetime import datetime, timedelta

# Calculate the time and date for calculations so far

xy = calcTimeStamp.calcTimeStamp()
start, end = (xy[0], xy[1])
start = str(start)
end = str(end)

In [100]:
import collections
import hashlib
import hmac
import time
from datetime import datetime
import requests
import json
import pprint

parameters = {
  "api-key": "vy8jbrjsxlbwgojepq3vfyfqfywyhvbd", 
  "api-secret": "sdqfm6wdfy9w0pqp2vdka38o6b4vcsvc",
  "station-id": 81211, 
  "end-timestamp": end,
  "start-timestamp": start,
  "t": int(time.time())
}

parameters = collections.OrderedDict(sorted(parameters.items()))

for key in parameters:
    print("Parameter name: \"{}\" has value \"{}\"".format(key, parameters[key]))

apiSecret = parameters["api-secret"];
parameters.pop("api-secret", None);

data = ""
for key in parameters:
    data = data + key + str(parameters[key])

print("Data string to hash is: \"{}\"".format(data))
print('\n')

"""
Calculate the HMAC SHA-256 hash that will be used as the API Signature.
"""
apiSignature = hmac.new(
  apiSecret.encode('utf-8'),
  data.encode('utf-8'),
  hashlib.sha256
).hexdigest()

"""
Let's see what the final API Signature looks like.
"""
print("API Signature is: \"{}\"".format(apiSignature))
print('\n')

# Building the URL to get the station

first_part = ('https://api.weatherlink.com/v2/historic/81211?')
api_key = ('api-key=vy8jbrjsxlbwgojepq3vfyfqfywyhvbd')
add_apisig = ('&api-signature=')
add_t = ('&t='+ str(int(time.time())))
start1 = "&start-timestamp=" + start
end1 = "&end-timestamp=" + end

#
URLfinal = (first_part + api_key + add_t + start1 + end1 + add_apisig + apiSignature)
print(URLfinal)

r =  requests.get(URLfinal)
r.encoding = 'utf-8'

r =  requests.get(URLfinal)

data_dir_file = '/Users/jameshayes/davis.json' 
with open(data_dir_file, "w") as fd:   
     json.dump(r.json(), fd)

Parameter name: "api-key" has value "vy8jbrjsxlbwgojepq3vfyfqfywyhvbd"
Parameter name: "api-secret" has value "sdqfm6wdfy9w0pqp2vdka38o6b4vcsvc"
Parameter name: "end-timestamp" has value "1679877205"
Parameter name: "start-timestamp" has value "1679803200"
Parameter name: "station-id" has value "81211"
Parameter name: "t" has value "1679877205"
Data string to hash is: "api-keyvy8jbrjsxlbwgojepq3vfyfqfywyhvbdend-timestamp1679877205start-timestamp1679803200station-id81211t1679877205"


API Signature is: "d3da1ea64098d1b9f17daf61e2169c152cc299f19a0ae5a36697055d72dc1685"


https://api.weatherlink.com/v2/historic/81211?api-key=vy8jbrjsxlbwgojepq3vfyfqfywyhvbd&t=1679877205&start-timestamp=1679803200&end-timestamp=1679877205&api-signature=d3da1ea64098d1b9f17daf61e2169c152cc299f19a0ae5a36697055d72dc1685


In [101]:
import collections
import hashlib
import hmac
import time
from datetime import datetime
import requests
import json
import pprint
import pytz
from dateutil.tz import tzutc,tzlocal

with open(data_dir_file) as fr:
    davisAPI = json.load(fr)
    
  
a = davisAPI['sensors']    
b = a[1]
c = (b['data'])
cLen = len(c)
print(cLen)

with open('/Users/jameshayes/testout.csv', 'w') as outfile: 
    i = 0
    while i < cLen:
        d = c[i]
        hi_temp = (d['temp_hi'])
        lo_temp = (d['temp_lo'])
        rainfall = (d['rainfall_in'])
        time = (d['ts'])
        ncR = 'None'
        
        date_time = datetime.fromtimestamp(time)
                        
        print(f'{hi_temp},{lo_temp},{rainfall},{ncR},{date_time}', file = outfile)
        i += 1

246


In [102]:
import pandas as pd

path_name = '/Users/jameshayes/'
file_name = 'testout.csv'
full_file = (f'{path_name}{file_name}')

df = pd.read_csv(full_file, index_col=False,names=['temp_hi', 'temp_lo', 'rainfall', 'fake_rain','time'])

In [103]:
import pandas as pd
from datetime import datetime
from dateutil.tz import tzutc, tzlocal

pd.set_option('display.max_rows', 1440)
pd.set_option('display.max_columns', 35)
pd.set_option('display.width', 1500)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

max_temp  = (df.sort_values(by='temp_hi', ascending=False))
max_T = max_temp.iloc[:1]
maxT = max_T['temp_hi'].values[0]
maxT = round(maxT)

min_temp  = (df.sort_values(by='temp_lo', ascending=True))
min_T = min_temp.iloc[:1]
minT = min_T['temp_lo'].values[0]
minT = round(minT)
minT = str(minT)
minT = minT.strip()

totR = df['rainfall'].sum()
totR = round(totR,2)
ncR = 'None'

recentT = (df['time'].iloc[-1])
datetime_obj = datetime.strptime(recentT, '%Y-%m-%d %H:%M:%S')
localT = datetime_obj.astimezone(tzlocal())
#lastTime = datetime_obj.strftime('%I:%M %p')
lastTime = localT.strftime('%I:%M %p')
print(lastTime)

# write the data to a csv file with an html suffix

with open('/Users/jameshayes/HiLoRain.csv', 'w') as outfile:
    print(f'{maxT},{minT},{totR},{ncR},{lastTime}',file = outfile)

08:30 PM
